In [1]:
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.stats.diagnostic import acorr_ljungbox
from scipy import stats
np.random.seed(1)

In [103]:
def model():
    ## Parameters of the model
    mm = 2 #switching parameter gamma in Brock Hommes
    pstar = 0.018
    # the central bank's inflation target
    eprational=1 # if all agents have rational forecast of inflation this parameter is 1
    epextrapol=0 # if all agents use inflation extrapolation this parameter is 1
    a1 = 0.5 #coefficient of expected output in output equation
    a2 = 0.2 #Interest elasticity of output demand
    a3 = 0.25 #Coefficient of public expenditure in output equation
    b1 = 0.5 #b1 is coefficient of expected inflation in inflation equation
    b2 = 0.05 #b2 is coefficient of output in inflation equation
    c1 = 1.5 #c1 is coefficient of inflation in Taylor equation
    c2 = 0.5 #c2 is coefficient of output in Taylor equation
    c3 = 0.5 #interest smoothing parameter in Taylor equation
    f1 = 0.6 #Public expenditure smoothing in fiscal rule
    f2 = 0.2 #Coefficient of output in fiscal rule
    f3 = 0.03 #Coefficient of public debt in fiscal rule
    rs = 0.01 #S.S. interest rate
    h1 = 0.4 #Coefficient of public expenditure in debt equation
    h2 = 1 #Coefficient of output in debt equation
    tau = 0.3 #Income tax rate
    rho=0.5 #Memory parameter
    pss = 0.01
    yss = 0.02
    A = np.array([[1,0,a2,-a3,0],
                  [-b2,1,0,0,0],
                  [-c2,-c1,1,0,0],
                  [0,0,0,1,f3],
                  [0,0,0,0,1]])    
    B = np.array([[a1,a2,0,0,0],
                  [0,b1,0,0,0],
                  [0,0,0,0,0],
                  [0,0,0,0,0],
                  [0,0,0,0,0]])
    C = np.array([0,0,-c1,0,0])
    D = np.array([[1-a1,0,0,0,0],
                  [0,1-b1,0,0,0],
                  [0,0,c3,0,0],
                  [-f2,0,0,f1,0],
                  [-h2*tau*(1+rs),-(1+rs),1,h1*(1+rs),1+rs]])
    T = 420
    TI = 300
    K = 50
    #length of period to compute divergence
    sigma = np.ones(5)*0.005
    epfs=pstar
    #forecast inflation targeters
    p = np.zeros(T)
    y = np.zeros(T)
    plagt = np.zeros(T)
    ylagt = np.zeros(T)
    r = np.zeros(T)
    epf = np.zeros(T)
    epc = np.zeros(T)
    ep = np.zeros(T)
    ey = np.zeros(T)
    CRp = np.zeros(T)
    FRp = np.zeros(T)
    alfapt = np.zeros(T)
    eyfunt = np.zeros(T)
    CRy = np.zeros(T)
    FRy = np.zeros(T)
    alfayt = np.zeros(T)
    anspirits = np.zeros(T)
    g = np.zeros(T) #government expenditures
    d = np.zeros(T) #deltas
    epsilont = np.zeros(T)
    etat = np.zeros(T)
    ut = np.zeros(T)
    nut = np.zeros(T)
    psit = np.zeros(T)
    
    GDP = 790

    #################################################
    #heuristic model
    #################################################
    alfap=0.5
    alfay=0.5
    K1=K+1

    for t in range(1,T):
        epsilont[t]=rho*epsilont[t-1]+sigma[0]*np.random.randn()#shocks in output equation [demand shock]
        etat[t]=rho*etat[t-1]+sigma[1]*np.random.randn()#shocks in inflation equation [supply shock]
        ut[t]=rho*ut[t-1]+sigma[2]*np.random.randn()#shocks in Taylor rule [interest rate shock]
        nut[t]=rho*nut[t-1]+sigma[3]*np.random.randn()
        psit[t]=rho*psit[t-1]+sigma[4]*np.random.randn()
        
        nut[312]=0.1
        if 313<=t<=324:
            nut[t]=0.1/12
            epsilont[t] = 0.1/12
        
        epsilon=epsilont[t]
        eta=etat[t]
        u=ut[t]
        nu = nut[t]
        psi = psit[t]
        
        
        shocks = np.array([epsilon,eta,u,nu,psi])
        epcs=p[t-1]
        if eprational==1:
            epcs=pstar

        eps=alfap*epcs+(1-alfap)*epfs
        if epextrapol==1:
            eps=p[t-1]
        # eps is expected inflation in next period
        eychar=y[t-1]
        eyfun=0+1/2
        eyfunt[t]=eyfun
        eys=alfay*eychar+(1-alfay)*eyfun
        forecast=np.array([eps, eys,0,0,0]) # expected inflation, expected y t+1
        ylag=y[t-1]
        plag=p[t-1]
        rlag=r[t-1]
        glag=g[t-1]
        dlag=d[t-1]
        lag=np.array([ylag,plag,rlag,glag,dlag])
        
        Z = np.dot(np.linalg.inv(A),(np.dot(B,forecast)+C*pstar+np.dot(D,lag)+shocks))

        
        y[t]=Z[0]
        p[t]=Z[1]
        r[t]=Z[2]
        g[t]=Z[3]
        d[t]=Z[4]
        #r[t]= (1-c3)*(c1*(p[t])+c2*y[t])+c3*rlag+u
        #if square==1
        #    r[t]=c1*[p[t]]^2+c2*y[t]+c3*r[t-1]+u
        #end
        plagt[t]=p[t-1]
        ylagt[t]=y[t-1]
        CRp[t]=rho*CRp[t-1]-(1-rho)*(epcs-p[t])**2
        FRp[t]=rho*FRp[t-1]-(1-rho)*(epfs-p[t])**2
        CRy[t]=rho*CRy[t-1]-(1-rho)*(eychar-y[t])**2
        FRy[t]=rho*FRy[t-1]-(1-rho)*(eyfun-y[t])**2

        alfap=np.exp(mm*CRp[t])/(np.exp(mm*CRp[t])+np.exp(mm*FRp[t]))
        alfay=np.exp(mm*CRy[t])/(np.exp(mm*CRy[t])+np.exp(mm*FRy[t]))
        alfapt[t]=alfap
        alfayt[t]=alfay

    # cut off initial stuff
    y = y[TI:]-np.median(y[:TI])
    p = p[TI:]-np.median(p[:TI])
    r = r[TI:]-np.median(r[:TI])
    d = d[TI:]-np.median(d[:TI])
    g = g[TI:]-np.median(g[:TI])
    
    # Now add to SS values
    g_SS = 0.363
    debt = np.exp(d+np.log(0.67))* GDP
    g = np.exp(g+np.log(g_SS)) * GDP
    r = np.exp(r+np.log(rs))*100
    p = np.exp(p+np.log(pss))*100
    y = np.exp(-y)*870
    
    
    return y,p,r,d,debt,g
_=model()


In [ ]:
Nruns = 500
outputs = np.zeros((6,Nruns,120))
for i in range(Nruns):
    ret = model()
    outputs[0,i,:] = ret[0]
    outputs[1,i,:] = ret[1]
    outputs[2,i,:] = ret[2]
    outputs[3,i,:] = ret[3]
    outputs[4,i,:] = ret[4]
    outputs[5,i,:] = ret[5]

In [ ]:
def plot_impulses(outputs):
    medians = np.array([np.median(w,axis=0) for w in outputs])
    upper_quartile = np.array([np.percentile(w,q=97.5,axis=0) for w in outputs])
    lower_quartile = np.array([np.percentile(w,q=2.5,axis=0) for w in outputs])
    y,p,r,d,debt,g = medians
    y_l,p_l,r_l,d_l,debt_l,g_l = lower_quartile
    y_u,p_u,r_u,d_u,debt_u,g_u = upper_quartile
    
    
    time = np.linspace(0,len(y)/12,len(y))
    fig, [[ax1,ax2],[ax3,ax4],[ax5,ax6]] = plt.subplots(3,2,figsize=(6,6))
    ax1.plot(time,y,color='tomato')
    ax1.fill_between(time,y_l,y_u,alpha=0.3,color='dodgerblue')
    ax1.set_title('GDP',y=0.8)
    ax1.set_ylabel('GDP (billion euro)')
    #ax1.set_ylabel('USD')
    ax2.plot(time,p,color='tomato')
    ax2.fill_between(time,p_l,p_u,alpha=0.3,color='dodgerblue')
    ax2.set_title('Inflation',y=0.8)
    ax2.set_ylabel('Inflation rate (%)')
    ax3.plot(time,r,color='tomato')
    ax3.fill_between(time,r_l,r_u,alpha=0.3,color='dodgerblue')
    ax3.set_ylabel('Interest rate (%)')
    ax3.set_title('Interest rate',y=0.8)
    ax4.plot(time,d,color='tomato')
    ax4.set_ylabel('Relative debt')
    ax4.fill_between(time,d_l,d_u,alpha=0.3,color='dodgerblue')
    ax4.set_title('Relative debt',y=0.8)
    ax5.plot(time,debt,color='tomato')
    ax5.fill_between(time,debt_l,debt_u,alpha=0.3,color='dodgerblue')
    ax5.set_title('Debt',y=0.8)
    ax5.set_ylabel('Debt (billion euro)')
    ax6.plot(time,g,color='tomato')
    ax6.fill_between(time,g_l,g_u,alpha=0.3,color='dodgerblue')
    ax6.set_title('Gov. Expenditure',y=0.8)
    ax6.set_ylabel('Expenses (billion euro)')
    
    for ax in [ax1,ax2,ax3,ax4,ax5,ax6]:
        ax.axvline(1,ls='--',color='tomato')
    for ax in [ax1,ax2,ax3,ax4]:
        ax.set_xticklabels(['']*len(ax.get_xticks()))
    for ax in [ax5,ax6]:
        ax.set_xlabel('Time since COVID (years)')
    plt.tight_layout(pad=2)
    plt.savefig('dummy_impulse_responses.pdf')
    plt.show()
    
plot_impulses(outputs)